In [1]:
#fghj

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

In [3]:
# you can switch the commented lines here to pick the other model

# inception resnet version 2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# You can choose ssd mobilenet version 2 instead and compare the results
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

In [4]:
model = hub.load(module_handle)

In [6]:
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction pruned(images) at 0x7F8E789DBD00>}))

In [11]:
detector = model.signatures['default']

In [7]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Fetches an image online, resizes it and saves it locally.
    
    Args:
        url (string) -- link to the image
        new_width (int) -- size in pixels used for resizing the width of the image
        new_height (int) -- size in pixels used for resizing the length of the image
        
    Returns:
        (string) -- path to the saved image
    '''
    
    
    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # opens the given URL
    response = urlopen(url)
    
    # reads the image fetched from the URL
    image_data = response.read()
    
    # puts the image data in memory buffer
    image_data = BytesIO(image_data)
    
    # opens the image
    pil_image = Image.open(image_data)
    
    # resizes the image. will crop if aspect ratio is different.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    
    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")
    
    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Image downloaded to %s." % filename)
    
    return filename

In [8]:
# You can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmpnsnw_ngq.jpg.


In [12]:
def load_img(path):
    '''
    Loads a JPEG image and converts it to a tensor.
    
    Args:
        path (string) -- path to a locally saved JPEG image
    
    Returns:
        (tensor) -- an image tensor
    '''
    
    # read the file
    img = tf.io.read_file(path)
    
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img


def run_detector(detector, path):
    '''
    Runs inference on a local file using an object detection model.
    
    Args:
        detector (model) -- an object detection model loaded from TF Hub
        path (string) -- path to an image saved locally
    '''
    
    # load an image tensor from a local file path
    img = load_img(path)

    # add a batch dimension in front of the tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # run inference using the model
    result = detector(converted_img)

    # save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])


In [13]:
# runs the object detection model and prints information about the objects found
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.65321755 0.6105044  0.6015257  0.5925565  0.59177715 0.58154935
 0.5505316  0.49575576 0.47424877 0.47322184 0.44066516 0.40511423
 0.39803788 0.39406568 0.3714853  0.3615593  0.3615071  0.34689158
 0.33362216 0.31252843 0.2887797  0.25758243 0.2574886  0.2519606
 0.24782136 0.23412377 0.20431975 0.20324676 0.17988248 0.17964779
 0.17374572 0.16431473 0.1603114  0.15895231 0.15620404 0.1546884
 0.1475448  0.13622677 0.12740056 0.12555611 0.1210271  0.11813046
 0.11387327 0.11229134 0.11129139 0.09718708 0.09137256 0.08976037
 0.08880259 0.08633586 0.08337478 0.0809518  0.07988787 0.07741443
 0.0773216  0.07631017 0.07507854 0.0738608  0.0723334  0.07204108
 0.07110199 0.06935615 0.068256   0.06427874 0.06248545 0.06226397
 0.06211134 0.05940052 0.05798776 0.05784459 0.05725625 0.05346687
 0.05304344 0.05242874 0.04892622 0.04813048 0.04578416 0.04424165
 0.04337565 0.04277812 0.04262631 0.04163568 0.04081763 0.03975999
 0.03945964 0.03944279 0.0386399  0.0376906  